<a href="https://colab.research.google.com/github/JefNtungila/DS-Unit-2-Classification-1/blob/master/DS_Sprint_Challenge_7_Classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science, Unit 2_
 
# Classification 1 Sprint Challenge: Predict Steph Curry's shots 🏀

For your Sprint Challenge, you'll use a dataset with all Steph Curry's NBA field goal attempts. (Regular season and playoff games, from October 28, 2009, through June 5, 2019.) 

You'll use information about the shot and the game to predict whether the shot was made. This is hard to predict! Try for an accuracy score in the high 50's or low 60's. The dataset was collected with the [nba_api](https://github.com/swar/nba_api) Python library.

In [5]:
!pip install category_encoders

     |████████████████████████████████| 92kB 5.9MB/s 


In [0]:
%matplotlib inline
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
pd.set_option('display.float_format', '{:.2f}'.format)

In [0]:
import pandas as pd
url = 'https://drive.google.com/uc?export=download&id=1fL7KPyxgGYfQDsuJoBWHIWwCAf-HTFpX'
df = pd.read_csv(url, parse_dates=['game_date']).set_index('game_date')
assert df.shape == (13958, 19)

In [41]:
df.head() #first look at table content

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,htm,vtm,season_type,scoremargin_before_shot
game_date,,,,,,,,,,,,,,,,,,,
2009-10-28,20900015,4,Stephen Curry,1,11,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,99,249,0,GSW,HOU,Regular Season,2.00
2009-10-28,20900015,17,Stephen Curry,1,9,31,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-122,145,1,GSW,HOU,Regular Season,0.00
2009-10-28,20900015,53,Stephen Curry,1,6,2,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,-60,129,0,GSW,HOU,Regular Season,-4.00
2009-10-28,20900015,141,Stephen Curry,2,9,49,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-172,82,0,GSW,HOU,Regular Season,-4.00
2009-10-28,20900015,249,Stephen Curry,2,2,19,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-68,148,0,GSW,HOU,Regular Season,0.00


In [42]:
df.describe()

,game_id,game_event_id,period,minutes_remaining,seconds_remaining,shot_distance,loc_x,loc_y,shot_made_flag,scoremargin_before_shot
count,13958.00,13958.00,13958.00,13958.00,13958.00,13958.00,13958.00,13958.00,13958.00,13958.00
mean,24428365.36,270.44,2.41,4.73,28.51,17.60,-0.55,131.26,0.47,1.62
std,7226619.76,169.93,1.13,3.33,17.60,10.30,124.72,102.67,0.50,10.13
min,20900015.00,2.00,1.00,0.00,0.00,0.00,-250.00,-41.00,0.00,-39.00
25%,21200912.00,109.00,1.00,2.00,13.00,8.00,-96.00,23.00,0.00,-4.00
50%,21500259.00,275.50,2.00,4.00,29.00,22.00,1.00,141.00,0.00,1.00
75%,21700963.00,398.00,3.00,7.00,44.00,25.00,95.00,219.00,1.00,8.00
max,41800403.00,752.00,6.00,11.00,59.00,83.00,247.00,811.00,1.00,43.00


In [43]:
df.dtypes

game_id                      int64
game_event_id                int64
player_name                 object
period                       int64
minutes_remaining            int64
seconds_remaining            int64
action_type                 object
shot_type                   object
shot_zone_basic             object
shot_zone_area              object
shot_zone_range             object
shot_distance                int64
loc_x                        int64
loc_y                        int64
shot_made_flag               int64
htm                         object
vtm                         object
season_type                 object
scoremargin_before_shot    float64
dtype: object

## Part 1. Prepare to model

### Required

1. **Do train/validate/test split.** Use the 2009-10 season through 2016-17 season to train, the 2017-18 season to validate, and the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your train set has 11081 observations, your validation set has 1168 observations, and your test set has 1709 observations.
2. **Begin with baselines for classification.** Your target to predict is `shot_made_flag`. What is the baseline accuracy for the validation set, if you guessed the majority class for every prediction?
3. **Use Ordinal Encoding _or_ One-Hot Encoding,** for the categorical features you select.
4. **Train a Random Forest _or_ Logistic Regression** with the features you select.


##Train Test Split

In [44]:
df.columns.values #getting the namesof the columns

array(['game_id', 'game_event_id', 'player_name', 'period',
       'minutes_remaining', 'seconds_remaining', 'action_type',
       'shot_type', 'shot_zone_basic', 'shot_zone_area',
       'shot_zone_range', 'shot_distance', 'loc_x', 'loc_y',
       'shot_made_flag', 'htm', 'vtm', 'season_type',
       'scoremargin_before_shot'], dtype=object)

In [0]:
df.index = pd.to_datetime(df.index) #needed to be able to split the data frame

In [52]:
train = df['2009-10-8':'2017-06-30']
val = df['2017-07-01':'2018-06-30']
test = df['2018-07-01':'2019-06-05']

print('train', train.shape)
print('val', val.shape)
print('test', test.shape)

train (11081, 19)
val (1168, 19)
test (1709, 19)


##Baseline for classification

In [53]:
y_train = train['shot_made_flag']
y_train.value_counts(normalize=True) #0 is the majority class

0   0.53
1   0.47
Name: shot_made_flag, dtype: float64

##One-hot encoding

In [57]:
train.describe(exclude='number').T.sort_values(by='unique') #checking which non numeric columns would be interesting

,count,unique,top,freq
player_name,11081,1,Stephen Curry,11081
shot_type,11081,2,2PT Field Goal,5941
season_type,11081,2,Regular Season,9642
shot_zone_range,11081,5,24+ ft.,5065
shot_zone_area,11081,6,Center(C),4820
shot_zone_basic,11081,7,Above the Break 3,4298
htm,11081,32,GSW,5512
vtm,11081,32,GSW,5569
action_type,11081,51,Jump Shot,5375


In [58]:
train['shot_zone_range'].value_counts(dropna=False) #shotzonerange chosen

24+ ft.            5065
Less Than 8 ft.    2711
16-24 ft.          2133
8-16 ft.           1097
Back Court Shot      75
Name: shot_zone_range, dtype: int64

In [110]:
encoder = ce.OrdinalEncoder() #shotzonerange info
encoded = encoder.fit_transform(train['shot_zone_range'])
encoded.head(10)
encoded['shot_zone_range'].dtype

dtype('int64')

In [96]:
train_encoded = encoder.fit_transform(train) #creating new dataframe for predictions
train_encoded.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,htm,vtm,season_type,scoremargin_before_shot
game_date,,,,,,,,,,,,,,,,,,,
2009-10-28,20900015,4,Stephen Curry,1,11,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),1,26,99,249,0,GSW,HOU,Regular Season,2.00
2009-10-28,20900015,17,Stephen Curry,1,9,31,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),2,18,-122,145,1,GSW,HOU,Regular Season,0.00
2009-10-28,20900015,53,Stephen Curry,1,6,2,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),3,14,-60,129,0,GSW,HOU,Regular Season,-4.00
2009-10-28,20900015,141,Stephen Curry,2,9,49,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),2,19,-172,82,0,GSW,HOU,Regular Season,-4.00
2009-10-28,20900015,249,Stephen Curry,2,2,19,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),2,16,-68,148,0,GSW,HOU,Regular Season,0.00


##Random Forest Model

In [97]:
train_encoded.columns.values

array(['game_id', 'game_event_id', 'player_name', 'period',
       'minutes_remaining', 'seconds_remaining', 'action_type',
       'shot_type', 'shot_zone_basic', 'shot_zone_area',
       'shot_zone_range', 'shot_distance', 'loc_x', 'loc_y',
       'shot_made_flag', 'htm', 'vtm', 'season_type',
       'scoremargin_before_shot'], dtype=object)

In [0]:
X_train = train_encoded[['game_id', 'game_event_id', 'player_name', 'period',
       'minutes_remaining', 'seconds_remaining', 'action_type',
       'shot_type', 'shot_zone_basic', 'shot_zone_area',
       'shot_zone_range', 'shot_distance', 'loc_x', 'loc_y',
        'htm', 'vtm', 'season_type',
       'scoremargin_before_shot']]

y_train = train_encoded['shot_made_flag']



In [101]:
val.columns.values

array(['game_id', 'game_event_id', 'player_name', 'period',
       'minutes_remaining', 'seconds_remaining', 'action_type',
       'shot_type', 'shot_zone_basic', 'shot_zone_area',
       'shot_zone_range', 'shot_distance', 'loc_x', 'loc_y',
       'shot_made_flag', 'htm', 'vtm', 'season_type',
       'scoremargin_before_shot'], dtype=object)

In [0]:
X_val =val[['game_id', 'game_event_id', 'player_name', 'period',
       'minutes_remaining', 'seconds_remaining', 'action_type',
       'shot_type', 'shot_zone_basic', 'shot_zone_area',
       'shot_zone_range', 'shot_distance', 'loc_x', 'loc_y',
        'htm', 'vtm', 'season_type',
       'scoremargin_before_shot']]

y_val = val['shot_made_flag']

In [112]:
%%time
from sklearn.ensemble import RandomForestClassifier

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='median'), 
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

# Fit on train, score on val
a = pipeline.fit(X_train, y_train)
# print('Validation Accuracy', pipeline.score(X_val, y_val))

CPU times: user 3.7 s, sys: 34.2 ms, total: 3.73 s
Wall time: 2.19 s


In [113]:
a

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['player_name', 'action_type', 'shot_type',
                                     'shot_zone_basic', 'shot_zone_area', 'htm',
                                     'vtm', 'season_type'],
                               drop_invariant=False, handle_missing='value',
                               handle_unknown='value', return_df=True,
                               use_cat_names=True, verbose=0)),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_valu...
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                    


### Stretch goals
Engineer at least 4 of these 5 features:

- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
- **Opponent**: Who is the other team playing the Golden State Warriors?
- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
- **Made previous shot**: Was Steph Curry's previous shot successful?

This Sprint Challenge has two parts. To demonstrate mastery on each part, do all the required, numbered instructions. To earn a score of "3" for the part, also do the stretch goals.


    

## Part 2. Evaluate models

### Required
1. Get your model's **validation accuracy.** (Multiple times if you try multiple iterations.)
2. Get your model's **test accuracy.** (One time, at the end.)
3. Get and plot your Random Forest's **feature importances** _or_ your Logistic Regression's **coefficients.**
4. Imagine this is the confusion matrix for a binary classification model. **Calculate accuracy, precision, and recall for this confusion matrix:**

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>


### Stretch goals
- Calculate F1 score for the provided, imaginary confusion matrix.
- Plot a real confusion matrix for your basketball model, with row and column labels.
- Print the classification report for your model.

##Confusion Matrix

In [119]:
Accuracy = (85 + 36 / (85+36+58+8))/100
Accuracy

0.8519251336898396

In [120]:
precision  = 85/(85+8)
precision

0.9139784946236559

In [122]:
recall = 85/(85+58)
recall

0.5944055944055944